# Import Dependencies and data 
#### Read the data using pandas 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import tree

In [ ]:
# Study data files
data1 = "../Data/nhl odds 2014-15.csv"
data2 = "../Data/nhl odds 2015-16.csv"
data3 = "../Data/nhl odds 2016-17.csv"
data4 = "../Data/nhl odds 2017-18.csv"
data5 = "../Data/nhl odds 2018-19.csv"
data6 = "../Data/nhl odds 2019-20.csv"

# Read the mouse data and the study results
data1 = pd.read_csv(data1)
data2 = pd.read_csv(data2)
data3 = pd.read_csv(data3)
data4 = pd.read_csv(data4)
data5 = pd.read_csv(data5)
data6 = pd.read_csv(data6)

data6 = data6.rename(columns={"PuckLine": "Puck Line", "OpenOU": "Open OU", "CloseOU": "Close OU"})

frames = [data1, data2, data3, data4, data5, data6]


# Merge all the different seasons 

In [ ]:
def set_win(row):
    if row['Point Dif'] > 0:
        return 1
    else:
        return 0

# Create row to see who won or loss by checking the point diferential was positive or negative.
def set_win_pl(row):
    if row['Point Dif'] >= 2:
        return 1
    elif (row['Point Dif'] >= -1) & (row['Puck Line'] == 1.5):
        return 1
    else:
        return 0

def data_clean(df):
    df = df.replace({'Team': {'Arizonas': 'Arizona', 'NewJersey': 'New Jersey', 'NYIslanders': 'NY Islanders', 'NYRangers': 'NY Rangers',  'SanJose': 'San Jose',  'St.Louis': 'St. Louis', 'TampaBay': 'Tampa Bay','LosAngeles':'Los Angeles'}})
    p_dif = []
    wins = []

    # Iterate through table and calculate point dif and winners and insert into list 
    for i in range(1, df.shape[0], 2):
        home_dif = df['Final'].iloc[i] - df['Final'].iloc[i-1]
        away_dif = df['Final'].iloc[i-1] - df['Final'].iloc[i]
        p_dif.append(away_dif)
        p_dif.append(home_dif)
            
    # Add point dif column and data         
    df['Point Dif'] = p_dif
    df = df.reset_index()
    # Create point differential column 
    line_move = []

    # Iterate through table and calculate point dif and insert into list 
    for i in range(len(df)):
        move = df['Open'].iloc[i] - df['Close'].iloc[i]
        line_move.append(move)

    # Add point dif column and data         
    df['Line Mvmt'] = line_move
    df = df.reset_index()
    
    # add the result column 
    df = df.assign(Win=df.apply(set_win, axis=1))
    df = df.assign(Win_PL=df.apply(set_win_pl, axis=1)) 
    
    goal_dict = {}
    net_goals = []
    wd = {}
    w = []    

    for i in df.iterrows():
    #point difference
        if i[1]['Team'] in goal_dict:
            net_goals.append(goal_dict[i[1]['Team']])
            goal_dict[i[1]['Team']] = goal_dict[i[1]['Team']] + int(i[1]['Point Dif'])
            w.append(wd[i[1]['Team']])
            wd[i[1]['Team']] = wd[i[1]['Team']] + int(i[1]['Win'])
        else:
            goal_dict[i[1]['Team']] = int(i[1]['Point Dif'])
            net_goals.append(0)
            wd[i[1]['Team']] = int(i[1]['Win'])
            w.append(0)
    
    df['season_wins'] = w        
    df['net_goals'] = net_goals
    return df

In [ ]:
new_frames = []
for frame in frames:
    clean = data_clean(frame)
    new_frames.append(clean)
    
df = pd.concat(new_frames)
cols = [0,1]
df.drop(df.columns[cols],axis=1,inplace=True)
df

In [ ]:
df.to_csv('data.csv')

# Cleaning Up the Dataframe

In [ ]:
cols = [0,1,3,4,5,6,7,8,10,12,13,14,15,16]
df.drop(df.columns[cols],axis=1,inplace=True)
df

In [ ]:
df = df.replace({'VH': {'V': 0, 'H': 1}})
df = df.rename(columns={"VH": "Home","Close": "ML", "Unnamed: 11": "PL"})
df = df[["Home", "ML", "PL", "Line Mvmt", "net_goals", "season_wins", "Win", "Win_PL"]]
df = df.dropna()

df

In [ ]:
df.to_csv('final_df.csv')

In [2]:
df = pd.read_csv('final_df.csv')
cols = [0]
df.drop(df.columns[cols],axis=1,inplace=True)
df

,Home,ML,PL,Line Mvmt,net_goals,season_wins,Win,Win_PL
0,0,-105,-305.0,0,0,0,1,1
1,1,-115,250.0,0,0,0,0,0
2,0,165,-180.0,0,0,0,0,1
3,1,-185,155.0,-10,0,0,1,0
4,0,130,-240.0,-29,0,0,1,1
...,...,...,...,...,...,...,...,...
15489,1,-146,170.0,-19,45,41,1,0
15490,0,145,-180.0,-5,-51,25,0,1
15491,1,-160,160.0,0,-35,28,1,0
15492,0,-200,115.0,23,30,41,1,1


# Random Forest Model

In [3]:
#target = df["Win_PL"]
target = df["Win"]
target_names = ["loss", "win"]

In [4]:
data = df.drop(columns=['Win', 'Win_PL'],axis=1)
feature_names = data.columns

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [180,190,200,210,220],
    'criterion': ['gini','entropy']
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, verbose=True, n_jobs=-1)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   48.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   58.8s finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [180, 190, 200, 210, 220]},
             verbose=True)

In [7]:
print(grid.best_params_)

{'criterion': 'entropy', 'n_estimators': 210}


In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=210,criterion='entropy')
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.5529168817759422

In [9]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.2192946781355956, 'net_goals'),
 (0.1993721232336998, 'Line Mvmt'),
 (0.19917100485432349, 'season_wins'),
 (0.18519142027159044, 'ML'),
 (0.17490344457169538, 'PL'),
 (0.022067328933095345, 'Home')]

# Deep Network

In [10]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=18, activation='relu', input_dim=6))
deep_model.add(Dense(units=9, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [12]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [13]:
deep_model.fit(
    X_train,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

Train on 11620 samples
Epoch 1/50
11620/11620 - 1s - loss: 1.1660 - accuracy: 0.5498
Epoch 2/50
11620/11620 - 1s - loss: 0.7451 - accuracy: 0.5598
Epoch 3/50
11620/11620 - 1s - loss: 0.7243 - accuracy: 0.5639
Epoch 4/50
11620/11620 - 1s - loss: 0.7159 - accuracy: 0.5628
Epoch 5/50
11620/11620 - 1s - loss: 0.7190 - accuracy: 0.5595
Epoch 6/50
11620/11620 - 1s - loss: 0.7076 - accuracy: 0.5695
Epoch 7/50
11620/11620 - 1s - loss: 0.7074 - accuracy: 0.5668
Epoch 8/50
11620/11620 - 1s - loss: 0.7009 - accuracy: 0.5662
Epoch 9/50
11620/11620 - 1s - loss: 0.7030 - accuracy: 0.5672
Epoch 10/50
11620/11620 - 1s - loss: 0.6930 - accuracy: 0.5728
Epoch 11/50
11620/11620 - 1s - loss: 0.6946 - accuracy: 0.5721
Epoch 12/50
11620/11620 - 1s - loss: 0.6914 - accuracy: 0.5710
Epoch 13/50
11620/11620 - 1s - loss: 0.6938 - accuracy: 0.5682
Epoch 14/50
11620/11620 - 1s - loss: 0.6911 - accuracy: 0.5698
Epoch 15/50
11620/11620 - 1s - loss: 0.6880 - accuracy: 0.5703
Epoch 16/50
11620/11620 - 1s - loss: 0.68

In [14]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

3874/3874 - 0s - loss: 0.6899 - accuracy: 0.5782
Deep Neural Network - Loss: 0.6899098299124466, Accuracy: 0.5782137513160706


In [ ]:
import pandas as pd
import re

In [ ]:
url = 'https://www.pro-football-reference.com/years/2007/games.htm'

In [ ]:
tables = pd.read_html(url)
tables